
# Delta Lake Uniform: Universal Format for all your Lakehouse tools


<img src="https://cms.databricks.com/sites/default/files/inline-images/image1_5.png" width="700px" style="float: right; margin-left: 50px"/>

Companies want to leverage open format and stay away from vendor lockin. Migration is costly and difficult, so they want to make the right decision up front and only have to save data once. 

They ultimately want the best performance at the cheapest price for all of their data workloads including ETL, BI, and AI, and the flexibility to consume that data anywhere.


Delta Universal Format (UniForm) automatically unifies table formats, without creating additional copies of data or more data silos. 

Teams that use query engines designed to work with Iceberg or Hudi data will be able to read Delta tables seamlessly, without having to copy data over or convert it. 

Customers don’t have to choose a single format, because **tables written by Delta will be universally accessible by Iceberg and Hudi readers.**


<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-engineering&org_id=796524194907820&notebook=%2F03-Delta-Lake-Uniform&demo_name=delta-lake&event=VIEW&path=%2F_dbdemos%2Fdata-engineering%2Fdelta-lake%2F03-Delta-Lake-Uniform&version=1">
<!-- [metadata={"description":"Quick introduction to Delta Lake. <br/><i>Use this content for quick Delta demo.</i>",
 "authors":["quentin.ambard@databricks.com"],
 "db_resources":{}}] -->

In [0]:
%run ./_resources/00-setup $reset_all_data=false

##![](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Leveraging Delta Lake power across all formats

<img src="https://cms.databricks.com/sites/default/files/inline-images/image3_2.png" style="float: right" width="650px" />

UniForm takes advantage of the fact that all three open lakehouse formats are thin layers of metadata atop Parquet data files. As writes are made, UniForm will incrementally generate this layer of metadata to spec for Hudi, Iceberg and Delta.

UniForm introduces negligible performance and resource overhead. 

We also saw improved read performance on UniForm-enabled tables relative to native Iceberg tables, thanks to Delta’s improved data layout capabilities .

With UniForm, customers can choose Delta with confidence, knowing that they’ll have broad support from any tool that supports lakehouse formats.

##  Create a UniForm-enabled Delta table

In [0]:
%sql
CREATE OR REPLACE TABLE user_uniform ( id BIGINT, firstname STRING, lastname STRING, email STRING)
    TBLPROPERTIES ('delta.universalFormat.enabledFormats' = 'iceberg', 
                   'delta.enableIcebergCompatV2' = 'true')

In [0]:
%sql
INSERT INTO user_uniform SELECT id, firstname, lastname, email FROM user_delta;  

SELECT * FROM user_uniform;

In [0]:
spark.sql("""
SHOW TBLPROPERTIES user_uniform
""").filter("key LIKE 'delta.universalFormat%' OR key LIKE 'delta_uniform_%'").display()

In [0]:
%sql
update user_uniform set firstname = 'John_c' where id < 100

In [0]:
%sql
describe extended user_uniform

Uniform delta tables are available as any other table. Because we enabled `iceberg` as format, each subsequential write will update the Delta and Iceberg metadata. <br>
Technically speaking, your table now contains 2 metadata folders:

* `delta_log` containing all Delta Lake format metadata
* `metadata` containing Iceberg format metadata


Your Delta Lake table is still available like any other table:

### Upgrade Existing Table to UniForm (Optional)



In [0]:
# If needed, upgrade legacy Delta tables using REORG
spark.sql("""
REORG TABLE user_uniform
 APPLY (UPGRADE UNIFORM(ICEBERG_COMPAT_VERSION=2)) """)

### Access your Delta Lake table using Iceberg REST Catalog API

If you're using an external storage, Databricks expose the table information through:

In [0]:
from databricks.sdk import WorkspaceClient
ws = WorkspaceClient() 
table_info = ws.api_client.do('GET', f'/api/2.1/unity-catalog/tables/{catalog}.{schema}.user_uniform')
table_info['delta_uniform_iceberg']

To read a managed table, you can leverage the Iceberg Catalog:

```
curl -X GET -H "Authentication: Bearer $OAUTH_TOKEN" -H "Accept: application/json" \
https://<workspace-instance>/api/2.1/unity-catalog/iceberg/v1/catalogs/<uc_catalog_name>/namespaces/<uc_schema_name>/tables/<uc_table_name>
```

You should then receive a response like this:

```
{
  "metadata-location": "s3://bucket/path/to/iceberg/table/metadata/file",
  "metadata": <iceberg-table-metadata-json>,
  "config": {
    "expires-at-ms": "<epoch-ts-in-millis>",
    "s3.access-key-id": "<temporary-s3-access-key-id>",
    "s3.session-token":"<temporary-s3-session-token>",
    "s3.secret-access-key":"<temporary-secret-access-key>",
    "client.region":"<aws-bucket-region-for-metadata-location>"
  }
}
```

For more details, see [the Uniform Documentation](https://docs.databricks.com/en/delta/uniform.html)

### Uniform Limitations

Note: Uniform works only with Unity Catalog and not HMS

Go to Limitations : https://docs.databricks.com/aws/en/delta/uniform#limitations



In [0]:
%sql
-- ALTER table user_uniform_dv set TBLPROPERTIES ('delta.columnMapping.mode' = 'name', 'delta.universalFormat.enabledFormats' = 'iceberg', 'delta.enableIcebergCompatV2' = 'true')

That's it! You can now access all your Delta Lake table as Iceberg table, while getting the power of Delta Lake and bazing fast queries with Liquid Clustering.

Your lakehouse is now fully open, without any vender lock-in. 


Next: Deep dive into Delta Lake Change Data Capture capability with [the 04-Delta-Lake-CDF notebook]($./04-Delta-Lake-CDF) or go back to [00-Delta-Lake-Introduction]($./00-Delta-Lake-Introduction).
